In [1]:
import networkx as nx
import time 
import pandas as pd
from utils.distance import get_distance
import matplotlib.pyplot as plt
import numpy as np
from utils.nn import nearest_neighbor
from utils.genalg import genetic_alg
from utils.simann import simulated_annealing
from utils.aco import ant_colony
from data import locations

Graph = nx.Graph()

df = pd.read_csv("calculations/pass_distances.csv")

for a in df.itertuples():
    Graph.add_edge(a.Loc1, a.Loc2, weight=a._4)
    
results = []
ITERATIONS = range(20)

In [2]:

# Nearest Neighbor
print("Running Nearest Neighbor")
start = time.time()
best_tour, best_distance = None, float('Inf')
for _ in ITERATIONS:
    tour, distance = nearest_neighbor(Graph, start_index=list(Graph.nodes).index('Bulacan Provincial Capitol Malolos'))
    if distance < best_distance:
        best_tour = tour
        best_distance = distance
nn_time = time.time() - start
print(f"NN Finished: {nn_time:.5f}s | Distance: {best_distance:.5f}m")

results.append(("Nearest Neighbor", nn_time, best_distance, best_tour))

Running Nearest Neighbor
NN Finished: 0.00179s | Distance: 110129.09446m


In [3]:
# Genetic Algorithm
print("Running Genetic Algorithm")
start = time.time()
best_tour, best_distance = None, float('Inf')
for _ in ITERATIONS:
    tour, distance = genetic_alg(Graph, 100, 0.01, 1000)
    if distance < best_distance:
        best_tour = tour
        best_distance = distance
ga_time = time.time() - start
print(f"GA Finished: {ga_time} | Distance: {best_distance}")
results.append(("Genetic Algorithm", ga_time, best_distance, best_tour))



Running Genetic Algorithm
GA Finished: 46.73311972618103 | Distance: 86906.42196309914


In [4]:

# Simulated Annealing
print("Running Simulated Annealing")
start = time.time()
best_tour, best_distance = None, float('Inf')
for _ in ITERATIONS:
    tour, distance = simulated_annealing(Graph, 10_000, 0.995,20_000)
    if distance < best_distance:
        best_tour = tour
        best_distance = distance
sa_time = time.time() - start
print(f"SA Finished: {sa_time:.5f}s | Distance: {best_distance:.5f}m")
results.append(("Simulated Annealing", sa_time, best_distance, best_tour))


Running Simulated Annealing
SA Finished: 1.19706s | Distance: 86893.75571m


In [5]:
ANTS, EPOCHS, ALPHA, BETA, RHO, Q = 50, 200, 1.0, 5.0, 0.1, 100

print("Running Ant Colony Optimization")
start = time.time()
best_tour, best_distance = None, float('Inf')
for _ in ITERATIONS:
    tour, distance = ant_colony(Graph, ANTS, EPOCHS, ALPHA, BETA, RHO, Q)
    if distance < best_distance:
        best_tour = tour
        best_distance = distance
ac_time = time.time() - start
print(f"SA Finished: {ac_time:.5f}s | Distance: {best_distance:.5f}m")
results.append(("Ant Colony Optimization", ac_time, best_distance, best_tour))

Running Ant Colony Optimization
SA Finished: 58.20961s | Distance: 87088.98394m


In [6]:
res_df = pd.DataFrame(results, columns=["Algorithm", "Time (s)", "Distance", "Tour"]).set_index("Algorithm")

In [27]:
res_df

,Time (s),Distance,Tour
Algorithm,,,
Nearest Neighbor,0.001795,110129.094459,"(Bulacan Provincial Capitol Malolos, Hiyas Bul..."
Genetic Algorithm,46.733120,86906.421963,"(St Francis of Aggigi Parish Meycauayan, Old M..."
Simulated Annealing,1.197061,86893.755712,"(St James the Apostle Parish Plaridel, Simbory..."
Ant Colony Optimization,58.209613,87088.983944,"(Simboryo Chapel of Quingua Plaridel, St James..."


In [26]:
remap = {
"St John of God Parish San Rafael": 'C0',
"Old Municipal Building Baliwag": 'C1',
"St Augustine Parish Baliwag": 'C2',
"St James the Apostle Parish Plaridel": 'C3',
"Simboryo Chapel of Quingua Plaridel": 'C4',
"Municipal Trial Court Pulilan": 'C5',
"Parish of St John the Baptist Calumpit": 'C6',
"Barasoain Church Malolos": 'C7',
"Immaculate Conception Parish Malolos": 'C8',
"Museum of Philippine Political History Malolos": 'C9',
"Malolos of Aguas Potables Malolos": 'C10',
"Hiyas Bulacan Cultural Center and Museum Malolos": 'C11',
"Bulacan Provincial Capitol Malolos": 'C12',
"Alberta Uitangcoy Santos House Malolos": 'C13',
"Dr Luis Santos House": 'C14',
"Old PNR  Guiguinto Station Guiguinto": 'C15',
"Constantino Ancestral HouseBalagtas": 'C16',
"Diocsan Shrine Bulakan": 'C17',
"Old Meycauayan PNR Station Meycauayan": 'C18',
"St Francis of Aggigi Parish Meycauayan": 'C19'
}

for i in [remap[i] for i in res_df.iloc[0]["Tour"]]:
    print(i,end=", ")

C12, C11, C7, C9, C8, C10, C14, C13, C3, C4, C5, C2, C1, C0, C15, C16, C17, C19, C18, C6, 

In [37]:
# Generate the Folium
import folium


for algo, time, distance, tour in res_df.itertuples():
    m = folium.Map(location=[locations[0].lat, locations[0].long], zoom_start=15)
    # Add marker to locations
    for loc in locations:
        folium.Marker(
            (loc.lat+0.002, loc.long)).add_to(m)
        folium.Marker(
            (loc.lat+0.002, loc.long),
            icon=folium.DivIcon(html=f"""
                <div style="display: inline-block; font-size: 12pt; color: white; font-weight: bold; background-color: blue; ">{remap[loc.name]}</div>
            """)
        ).add_to(m)
    #Add edges 
    for i in range(len(tour)):
        loc1 = tour[i]
        loc2 = tour[(i + 1) % len(tour)]

        loc1 = next((l for l in locations if l.name == loc1))
        loc2 = next((l for l in locations if l.name == loc2))
        folium.PolyLine([(loc1.lat, loc1.long), (loc2.lat, loc2.long)], color="blue", weight=5, opacity=0.8).add_to(m)

    print(f"Saving file to {algo}.html...")
    m.save(f"calculations/{algo}.html")

Saving file to Nearest Neighbor.html...
Saving file to Genetic Algorithm.html...
Saving file to Simulated Annealing.html...
Saving file to Ant Colony Optimization.html...
